# Bias Uncovering Test Case (BTC) for Gender Bias

In [1]:
import os
import pandas as pd
import numpy as np
import math
import time
import pickle

from metric import evaluate_btc, evaluate_btc_mtnlp
from utils import accuracy
from utils import load_mutant_and_prediction

## Measuring the number of BTCs found by BiasFinder

In [2]:
mutation_tool = "biasfinder"
bias_type = "gender"
task = "imdb"  # dataset used for fine-tuning
model = "bert-base-uncased"
mutant = "imdb"  # dataset used for generating mutants


df = load_mutant_and_prediction(mutation_tool, model, bias_type, mutant)

df.head()


,label,mutant,template,original,gender,template_id,prediction,original_prediction
0,0,"I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...",male,929,0,0
1,0,"I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...",female,929,0,0
2,1,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,male,1980,1,1
3,1,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,male,1980,1,1
4,1,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,male,1980,1,1


In [3]:
print("Accuracy on mutants: {:.2f}%".format(accuracy(df["label"],df["prediction"])))


Accuracy on mutants: 93.29%


In [4]:
def print_evaluation(evaluation):
    print("# Mutants \t:", evaluation["mutant"])
    print("# Templates \t:", evaluation["template"])
    print("# BTCs \t\t:", evaluation["btc"])

evaluation = evaluate_btc(df["label"], df["prediction"], df["mutant"], df["template"], "gender", df["gender"])
print_evaluation(evaluation)

# Mutants 	: 153866
# Templates 	: 3015
# BTCs 		: 5543


## IMDB

Measure the number of bias uncovering test cases found by each tool, i.e. BiasFinder, MT-NLP, and EEC, on models fine-tuned using IMDB dataset.

In [10]:
models = ["bert-base-uncased", "bert-base-cased", "roberta-base", "xlnet-base-cased",
          "albert-base-v2", "microsoft/mpnet-base", 
          "facebook/muppet-roberta-base", "google/electra-base-generator"]

mutation_tools = ["biasfinder", "mtnlp", "eec"]
bias_type = "gender"
task = "imdb"  # dataset used for fine-tuning
mutant = "imdb"  # dataset used for generating mutants

d = pd.DataFrame(columns=["model", "tool",
                          "template", "mutant", "btc"])

for model in models:
    for mutation_tool in mutation_tools:
        df = load_mutant_and_prediction(
            mutation_tool, model, bias_type, mutant)
        if mutation_tool == "mtnlp" :
            evaluation = evaluate_btc_mtnlp(df)
        else : 
            evaluation = evaluate_btc(df["label"], df["prediction"],
                                  df["mutant"], df["template"], "gender", df["gender"])

        d = d.append(
            {
                "model": model,
                "tool": mutation_tool,
                "template": evaluation['template'],
                "mutant": evaluation['mutant'],
                "btc": evaluation['btc']
            },
            ignore_index=True)

d


,model,tool,template,mutant,btc
0,bert-base-uncased,biasfinder,3015,153866,5543
1,bert-base-uncased,mtnlp,95219,285976,762
2,bert-base-uncased,eec,140,8400,8828
3,bert-base-cased,biasfinder,3015,153866,7723
4,bert-base-cased,mtnlp,95219,285976,1447
5,bert-base-cased,eec,140,8400,5674
6,roberta-base,biasfinder,3015,153866,8051
7,roberta-base,mtnlp,95219,285976,779
8,roberta-base,eec,140,8400,4560
9,xlnet-base-cased,biasfinder,3015,153866,2897


## Twitter Sentiment 140

Measure the number of bias uncovering test cases found by each tool, i.e. BiasFinder, MT-NLP, and EEC, on models fine-tuned using Twitter Sentiment 140 dataset.

In [11]:
models = ["bert-base-uncased", "bert-base-cased", "roberta-base", "xlnet-base-cased",
          "albert-base-v2", "microsoft/mpnet-base",
          "facebook/muppet-roberta-base", "google/electra-base-generator"]

mutation_tools = ["biasfinder", "mtnlp", "eec"]

bias_type = "gender"
task = "twitter_s140"  # dataset used for fine-tuning
mutant = "twitter_s140"  # dataset used for generating mutants

d = pd.DataFrame(columns=["model", "tool",
                          "template", "mutant", "btc"])

for model in models:
    for mutation_tool in mutation_tools:
        df = load_mutant_and_prediction(
            mutation_tool, model, bias_type, mutant)
        if mutation_tool == "mtnlp":
            evaluation = evaluate_btc_mtnlp(df)
        else:
            evaluation = evaluate_btc(df["label"], df["prediction"],
                                      df["mutant"], df["template"], "gender", df["gender"])

        d = d.append(
            {
                "model": model,
                "tool": mutation_tool,
                "template": evaluation['template'],
                "mutant": evaluation['mutant'],
                "btc": evaluation['btc']
            },
            ignore_index=True)

d


,model,tool,template,mutant,btc
0,bert-base-uncased,biasfinder,1769,63104,12389
1,bert-base-uncased,mtnlp,15150,59462,672
2,bert-base-uncased,eec,140,8400,1274
3,bert-base-cased,biasfinder,1769,63104,14373
4,bert-base-cased,mtnlp,15150,59462,793
5,bert-base-cased,eec,140,8400,238
6,roberta-base,biasfinder,1769,63104,29167
7,roberta-base,mtnlp,15150,59462,886
8,roberta-base,eec,140,8400,186
9,xlnet-base-cased,biasfinder,1769,63104,16711
